In [1]:
%run Functions.py

In [2]:
def ComputeBackSpaceActivations(KeysSelected, Event_ind, PhraseUser):
    
    KeysSelectedKey = [val[1] for val in KeysSelected]
    
    nBackSpaceActivationsList = list()
    
    for ind, eventStartInd in enumerate(Event_ind['start']):
        eventEndInd = Event_ind['end'][ind]
        KeysSelectedTrial = KeysSelectedKey[eventStartInd:eventEndInd]
        
        nBackSpaces = len([key for key in KeysSelectedTrial if key == 'BackOne' or key == 'BackMany'])
        
        nBackSpaceActivationsList.append(nBackSpaces/len(PhraseUser[ind][1]))
        
    
    return nBackSpaceActivationsList

In [4]:
metricComputed = 'BackSpaceActivations'

dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

resultFileName = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Performance\Subject_Block_Session_Trial_' + metricComputed +  '.xlsx'



for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
        if 'Picture' in root:
            continue
        #if 'af\\2\\2019-01-17-15-03-40_1' not in root:
        #    continue
            
        scratchPad = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerScratchPad = csv.reader(fScratchPad, quotechar=None)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
                
            if fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
                    
                     
        if scratchPad is None or userKeys is None or phraseLog is None:
            continue
        else:
            print('subject path', root)
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            # fix scratchpad due to comma related file changes
            scratchPad_new = FixScratchPad(scratchPad)
            
            # need to fix keys selected, where rows get combined because of an inverted comma
            keysSelected_new = FixKeysSelected(keysSelected)
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
                
            # find dwell time of typing
            userKeys_wDwellTime = ComputeDwellTime(userKeys_new, root)
            
            # if it is the 2nd part of the session, picture is not described:
            if '2ndPart' in root:
                picture = 'not_described'
            else:
                picture = 'described'
            
            phraseUserEnd_reduced = scratchPadPhraseEdit(scratchPad_new, subjName, root, picture)
            
            # find start time of typing
            timeTyping = OptiKeyTypingTime(userKeys_new)
            
            # divide complete data into epochs of phrases
            timeStartEndKeys_reading, timeStartEndKeys_writing = FindExptStartEndTimes(keysSelected_new, timeTyping, root)
            
            # use the times of trial start and end, to divide the time of trial into reading and writing
            eventTrialsInKeysSelected_reading, eventTrialsInKeysSelected_readingIndex = \
            FindReadingPartsOfTrial_inKeysSelected(timeStartEndKeys_reading, keysSelected_new, root, userKeys_wDwellTime)
            
            eventTrialsInKeysSelected_writing, eventTrialsInKeysSelected_writingIndex = \
            FindWritingPartsOfTrial_inKeysSelected(timeStartEndKeys_writing, keysSelected_new, eventTrialsInKeysSelected_reading)
            
            
            
            # Find letters not selected:
            # Find all keys that are not selected, just shortly looked at 
            nBackSpaceActivations = ComputeBackSpaceActivations(keysSelected_new, eventTrialsInKeysSelected_writingIndex,\
                                                                phraseUserEnd_reduced)
            
            if '1stPart' in root:
                print('1stPart')
                nBackSpaceActivations1 = nBackSpaceActivations 
                continue
            
            
            
            if '2ndPart' in root:
                print('2ndPart')
                nBackSpaceActivations2 = nBackSpaceActivations
                
                nBackSpaceActivations = nBackSpaceActivations1 + nBackSpaceActivations2
                
                nBackSpaceActivations1 = list()
                
                
            dataToSave = DataForEveryTrial()
            dataToSave.subjectID = subjAndSessionName.split('__')[0]
            dataToSave.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave.sessionNumber = subjAndSessionName[-1]
            dataToSave.variable = metricComputed
            dataToSave.dataForTrial = nBackSpaceActivations
            dataToSave.resultPathName = resultFileName
            
            print(dataToSave.printInfo())
            #dataToSave.AddToFile()
            

subject path E:\Data\Data\ac\1\2019-02-11-11-18-30_1
subject and session name:  ac__1__2019-02-11-11-18-30_1
  subjectID block session  trial  BackSpaceActivations
0        ac     1       1      0              0.015152
1        ac     1       1      1              0.022727
2        ac     1       1      2              0.000000
3        ac     1       1      3              0.000000
4        ac     1       1      4              0.000000
subject path E:\Data\Data\ac\1\2019-02-11-11-33-32_2
subject and session name:  ac__1__2019-02-11-11-33-32_2
  subjectID block session  trial  BackSpaceActivations
0        ac     1       2      0              0.113208
1        ac     1       2      1              0.025907
2        ac     1       2      2              0.005780
3        ac     1       2      3              0.019608
4        ac     1       2      4              0.036810
subject path E:\Data\Data\ac\2\2019-02-12-15-00-04_1
subject and session name:  ac__2__2019-02-12-15-00-04_1
  subjectID b

KeyboardInterrupt: 